In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ディレクトリ移動
import os
comp_name = "tabular-playground-series-feb-2022"
root_dir_name = '/content/drive/Othercomputers/macbook-air/'
comp_dir_name = "TPS-Feb2022"
dir_path = f'{root_dir_name}{comp_dir_name}/notebooks'
os.chdir(dir_path)

In [ ]:
# パスの定義
import yaml
CONFIG_FILE = '../configs/config.yaml'

with open(CONFIG_FILE) as file:
    yml = yaml.load(file)

MODEL_DIR_NAME = yml['SETTING']['MODEL_DIR_NAME']
FEATURE_DIR_NAME = yml['SETTING']['FEATURE_DIR_NAME']
RAW_DIR_NAME = yml['SETTING']['RAW_DIR_NAME']
SUB_DIR_NAME = yml['SETTING']['SUB_DIR_NAME']


os.environ['KAGGLE_CONFIG_DIR'] = f'{root_dir_name}{comp_dir_name}'

In [ ]:
import pandas as pd
import joblib

## pickleファイルの読み込み

In [ ]:
# pd.read_pickle(FEATURE_DIR_NAME + 'umap_output_train.pkl')
# pd.read_pickle(FEATURE_DIR_NAME + 'umap_output_test.pkl')

In [ ]:

path = MODEL_DIR_NAME + 'lgb_0226_0545/lgb_0226_0545-fold1.model'
model = joblib.load(path)

In [ ]:
class test():
  def __init__(self):
    print(self.__class__.__name__)

# test()

In [ ]:
a.__class__()